In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
from sklearn.feature_selection import f_classif
#import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
train_df = pd.read_csv('/Users/mengdili/Fall2018/ml539/train_df.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.



In [3]:
train_df.drop('Unnamed: 0',axis=1,inplace=True,errors='ignore')
# train_df.head()

In [4]:
copy_train_df = train_df.copy()

In [5]:
const_cols = [c for c in copy_train_df.columns if copy_train_df[c].nunique(dropna=False)==1 ]
cols_to_drop = const_cols + ['sessionId'] + ['visitId']+["trafficSource.campaignCode"]+['fullVisitorId']

copy_train_df = copy_train_df.drop(cols_to_drop, axis=1)

In [6]:
copy_train_df["totals.transactionRevenue"].fillna(0, inplace=True)
copy_train_df["totals.pageviews"].fillna(0, inplace=True)

# change date to get only month
def changeDateToMonth(ts):
    return str(ts)[4:6]
copy_train_df['date'] = copy_train_df['date'].apply(changeDateToMonth)

# changing visitStartTime from POSIX timestamp to hour
def changePOSIXtoHour(ts):
    return datetime.utcfromtimestamp(ts).strftime('%H')
copy_train_df['visitStartTime'] = copy_train_df['visitStartTime'].apply(changePOSIXtoHour)

# replace 'totals.bounces' : 1->0, nan->1
def switch(ts):
    if ts == 1:
        return 0
    else:
        return 1
copy_train_df['totals.bounces'] = copy_train_df['totals.bounces'].apply(switch)

def isNewVisit(val):
    if(val != 1):
        return 0
    else:
        return 1
copy_train_df['totals.newVisits'] = copy_train_df['totals.newVisits'].apply(isNewVisit)

def isRevenue(val):
    if (val >0):
        return 1
    else:
        return 0
copy_train_df['isRevenue'] = copy_train_df['totals.transactionRevenue'].apply(isRevenue) 
copy_train_df['isRevenue'] = copy_train_df['isRevenue'].astype(str)

In [7]:
copy_train_df['trafficSource.adwordsClickInfo.page'].fillna(0, inplace=True)
cols = ['trafficSource.adwordsClickInfo.adNetworkType','trafficSource.adContent','trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.isVideoAd','trafficSource.adwordsClickInfo.slot',
       'trafficSource.isTrueDirect','trafficSource.keyword','trafficSource.referralPath']
for col in cols:
    copy_train_df[col].fillna('missing', inplace=True)

In [8]:
copy_train_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
channelGrouping,903653,8,Organic Search,381561,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,903653,12,11,113972,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitNumber,903653,NaN,NaN,NaN,2.2649,9.28373,1,1,1,1,395
visitStartTime,903653,24,18,51096,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device.browser,903653,54,Chrome,620364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device.deviceCategory,903653,3,desktop,664479,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device.isMobile,903653,2,False,664530,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device.operatingSystem,903653,20,Windows,350072,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.city,903653,649,not available in demo dataset,508229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.continent,903653,6,Americas,450377,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
num = ['visitNumber','totals.bounces','totals.hits','totals.newVisits','totals.pageviews',
       'totals.transactionRevenue','trafficSource.adwordsClickInfo.page']
one_zero = ['device.isMobile','trafficSource.adwordsClickInfo.isVideoAd','trafficSource.isTrueDirect',
           'isRevenue']
label = ['date','visitStartTime']
onehot = ['device.deviceCategory','trafficSource.adwordsClickInfo.adNetworkType',
         'trafficSource.adwordsClickInfo.slot']
hashing = ['channelGrouping','geoNetwork.continent','device.browser','device.operatingSystem','geoNetwork.city','geoNetwork.country',
          'geoNetwork.metro','geoNetwork.networkDomain','geoNetwork.region','geoNetwork.subContinent',
          'trafficSource.adContent','trafficSource.adwordsClickInfo.gclId','trafficSource.keyword',
          'trafficSource.referralPath','trafficSource.source','trafficSource.campaign','trafficSource.medium']


In [10]:
copy2_train_df = copy_train_df.copy()

In [11]:
# one_zero
def switch1(ts):
    if ts == True:
        return 1
    else:
        return 0
copy2_train_df['device.isMobile'] = copy2_train_df['device.isMobile'].apply(switch1)
# copy2_train_df['device.isMobile'] = copy2_train_df['device.isMobile'].astype(str)

def switch2(ts):
    if ts == 'missing':
        return 0
    else:
        return 1
copy2_train_df['trafficSource.adwordsClickInfo.isVideoAd'] = copy2_train_df['trafficSource.adwordsClickInfo.isVideoAd'].apply(switch2)
# copy2_train_df['trafficSource.adwordsClickInfo.isVideoAd'] = copy2_train_df['trafficSource.adwordsClickInfo.isVideoAd'].astype(str)
copy2_train_df['trafficSource.isTrueDirect'] = copy2_train_df['trafficSource.isTrueDirect'].apply(switch2)
# copy2_train_df['trafficSource.isTrueDirect'] = copy2_train_df['trafficSource.isTrueDirect'].astype(str)

In [12]:
#label
# np.unique(copy2_train_df['date'])
date_ord_map = {'01': 1, '02': 2, '03': 3, '04': 4, '05': 5, '06': 6,'07': 7,'08': 8,
               '09': 9,'10': 10,'11': 11,'12': 12}
copy2_train_df['date'] = copy2_train_df['date'].map(date_ord_map)

# np.unique(copy2_train_df['visitStartTime'])
time_ord_map = {'00':0,'01': 1, '02': 2, '03': 3, '04': 4, '05': 5, '06': 6,'07': 7,'08': 8,
               '09': 9,'10': 10,'11': 11,'12': 12,'13': 13, '14': 14, '15': 15, '16': 16,
                '17': 17, '18': 18,'19': 19,'20': 20,'21': 21,'22': 22,'23': 23}
copy2_train_df['visitStartTime'] = copy2_train_df['visitStartTime'].map(time_ord_map)

In [13]:
copy3 = copy2_train_df.copy()
# copy3.describe(include='all').T

In [14]:
#onehot
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
# transform and map pokemon generations
lbe = LabelEncoder()
ohe = OneHotEncoder()
copy3_ohe = copy3.copy()
# copy3_ohe.drop(onehot, axis=1)
for col in onehot:
    labels = lbe.fit_transform(copy3[col])
    copy3_ohe[col] = labels
# encode generation labels using one-hot encoding scheme
    feature_arr = ohe.fit_transform(
                                  copy3_ohe[[col]]).toarray()
    feature_labels = list(str(col)+'_'+str(cls_label) for cls_label in lbe.classes_)
    features = pd.DataFrame(feature_arr,columns=feature_labels)
    copy3_ohe = pd.concat([copy3_ohe, features], axis=1)

copy3_ohe = copy3_ohe.drop(onehot,axis=1)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning:

The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncod

In [14]:
copy3_ohe.describe(include='all').T
# copy3_ohe.columns

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,903653,NaN,NaN,NaN,6.98767,3.48458,1,4,7,10,12
visitNumber,903653,NaN,NaN,NaN,2.2649,9.28373,1,1,1,1,395
visitStartTime,903653,NaN,NaN,NaN,12.5292,6.82545,0,7,14,18,23
device.browser,903653,54,Chrome,620364,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device.isMobile,903653,NaN,NaN,NaN,0.264618,0.44113,0,0,0,1,1
device.operatingSystem,903653,20,Windows,350072,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.city,903653,649,not available in demo dataset,508229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.country,903653,222,United States,364744,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.metro,903653,94,not available in demo dataset,508229,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork.networkDomain,903653,28064,(not set),244881,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
copy_ohe = copy3_ohe.copy()

In [19]:
from sklearn.feature_extraction import FeatureHasher

fh = FeatureHasher(n_features=5, input_type='string')
for col in hashing:
    hashed_arr = fh.fit_transform(copy_ohe[col]).toarray()
#     hashed_features = hashed_features.toarray()
    feature_labels = list(str(col)+'_'+str(i) for i in range(5))
    hashed_features = pd.DataFrame(hashed_arr,columns=feature_labels)
    copy_ohe = pd.concat([copy_ohe, hashed_features], axis=1)

In [20]:
copy_ohe = copy_ohe.drop(hashing,axis =1)

In [21]:
copy_ohe.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,903653,NaN,NaN,NaN,6.98767,3.48458,1,4,7,10,12
visitNumber,903653,NaN,NaN,NaN,2.2649,9.28373,1,1,1,1,395
visitStartTime,903653,NaN,NaN,NaN,12.5292,6.82545,0,7,14,18,23
device.isMobile,903653,NaN,NaN,NaN,0.264618,0.44113,0,0,0,1,1
totals.bounces,903653,NaN,NaN,NaN,0.501324,0.499999,0,0,1,1,1
totals.hits,903653,NaN,NaN,NaN,4.59654,9.64144,1,1,2,4,500
totals.newVisits,903653,NaN,NaN,NaN,0.77802,0.415578,0,1,1,1,1
totals.pageviews,903653,NaN,NaN,NaN,3.84934,7.025,0,1,1,4,469
totals.transactionRevenue,903653,NaN,NaN,NaN,1.70427e+06,5.27787e+07,0,0,0,0,2.31295e+10
trafficSource.adwordsClickInfo.isVideoAd,903653,NaN,NaN,NaN,0.0237481,0.152263,0,0,0,0,1


In [19]:
arr = ['isRevenue','totals.transactionRevenue']
X_train = copy_ohe.drop(arr, axis=1)
YReg = copy_ohe['totals.transactionRevenue']
YCla = copy_ohe['isRevenue']

In [20]:
X_train.to_csv('X_train.csv')
YReg.to_csv('YReg_train.csv')
YCla.to_csv('YCla_train.csv')

In [22]:
copy_ohe.to_csv('encodered_train.csv')